To-do:
1. Текст был очищен только от одного мусорного элемента в качестве примера. Исслудйте данные через ноутбук или чере веб-интерфейс BigQuery на предмет других мусорных элементов в тексте, которые не несут в себе никакого особого смысла, а только создают шум в данных. Доработайте функцию очистки тектосвых данных, чтобы в нее можно было передать список ненужного мусора и разом выполнялась очистка
2. Проведите стратифицировнную кросс-валидуцию нейросетевого классификатора https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
3. Поэксперементируйте с гиперпараметрами нейросетевого классификатора, постарайтесь повысить качество его работы
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4. Попробуйте использовать не Word2Vec для получения векторого представления текста, а TF-IDF преобразование http://zabaykin.ru/?p=558 http://nlpx.net/archives/57
5. Попробуйте использовать более тонко настриваемые алгоритмы нейросетей, например из этого видео https://www.youtube.com/watch?v=cPkH1k3U1c8 

6. Реверс инжинеринг по Толмачевски

In [126]:
import warnings
warnings.filterwarnings('ignore')

from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt

from langdetect import detect
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix
from gensim.models.word2vec import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt
import collections as collect
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [127]:
CREDENTIALS = service_account.Credentials.from_service_account_info({
***
})

In [128]:
#funtion for getting fresh data from DWH for workload model
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
[example]
Input: Credentials = credentials_object
Output: description	                                        channel	 category	category_flag
        \nChat transcript:\nVisitor: I want to buy wit...	chat	 ps	        1
        \nChat transcript:\nVisitor: hell i had a prob...	chat	 ps	        1
        \nChat transcript:\nVisitor: لا استطيع الشراء ...	 chat	  ps	     1
"""
def getDwhData(Credentials):
    statement_bigquery_sql = " ".join(["select description, channel, case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 'ps'",
                                       "else 'other'",
                                       "end as category,",
                                       "case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 0",
                                       "else 1",
                                       "end as category_flag",
                                       "from `xsolla_summer_school.customer_support`",
                                       "where manual_category is not null and",
                                       "manual_category <> '' and",
                                       "description is not null and",
                                       "description <> '' and",
                                       "channel is not null and",
                                       "channel <> '' and",
                                       "channel in ('chat','facebook')"])
    
    dataframe_bigquery = pandas_gbq.read_gbq(statement_bigquery_sql,project_id='findcsystem', credentials=Credentials, dialect='standard')

    return dataframe_bigquery


"""[summary]
Function for transform text to lower case
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["text_1","text_2"]
"""
def lowerCase(Corpus):
    corpus = [i.lower().replace('\n','') for i in Corpus]
    return corpus


"""[summary]
Function for getting language of text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["en","ru"]
"""
def getTextLanguage(Corpus):
    txt_lang = []
    for txt in Corpus:
        try:
            lang = detect(txt)
            txt_lang.append(lang)
        except:
            lang = 'error'
            txt_lang.append(lang)
    
    return txt_lang


"""[summary]
Function for tokenization text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["word1 word2","word3 word4"]
Output: [["word1","word2"],["word3","word4"]]
"""  
def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 


"""[summary]
Function for clear text after garbage
[description]
Corpus - list or array object, with text data
Substr - string, regular expression
[example]
Input: Corpus = [["word1","word2"],["word3","word4"]]
       Substr = r'word1
Output: [["word2"],["word3","word4"]]
"""  
def clearTextAfterGarbage(Corpus, Substrlist):
    clear_corpus = []
    cleared_corpus = Corpus
    for text in cleared_corpus:
        indexes = []
        text_len = len(text)
        for sub in Substrlist:
            try:
                for i in range(0, text_len):
                    res = re.search(sub, text[i])
                    if res != None:
                        indexes.append(i)

                #delete garbage word from text
                for index in indexes:
                    del text[index]

                clear_corpus.append(text)
            except:
                clear_corpus.append("error")
        
        cleared_corpus = clear_corpus
    return cleared_corpus

"""[summary]
Function for clear text from garbage
[description]
wordlist - list or array or series of lists or arrays with tokenazed words
garbagelist - list of words that needs to be cleared
[example]
Input: wordlist = [["word1","word2"],["word3","word4"]]
       garbagelist = ["word1", "word4"]
Output: [["word2"],["word4"]]
"""  
def FastClear(wordlist, garbagelist): # Среднебыстрая функция очистки мусора
    wordlist = wordlist.tolist()
    clearing_text = []
    i = 0
    for text in wordlist:
        i+=1
        text1 = [x for x in text if x not in garbagelist]
        if len(text1) == 0:
            text1.append('empty')
        clearing_text.append(text1)
        if i % 500 == 0:
            print(f"Done: {i} out of {len(wordlist)}")
           
    return clearing_text
        
"""[summary]
Function for clear text from garbage
[description]
wordlist - list or array or series of lists or arrays with tokenazed words
garbagelist - list of words that needs to be cleared
[example]
Input: wordlist = [["word1","word2"],["word3","word4"]]
       garbagelist = ["word1", "word4"]
Output: [["word2"],["word4"]]
"""  

def FastClear2(wordlist, garbagelist): # Очень быстрая функция очистки мусора
    clearing_text = []
    i = 0
    for text in wordlist:
        i+=1
        text1 = set(text) - set(garbagelist)
        if len(text1) == 0:
            list(text1).append('empty')
        clearing_text.append(text1)
        if i % 500 == 0:
            print(f"Done: {i} out of {len(wordlist)}")
    return clearing_text                        
            

"""[summary]
Build word vector by using pre-trained Word2Vec model
[description]
Size - lenght of vector
Word2Vec_Model - gensim object
"""  
def buildWordVector(Text,Size,Word2Vec_Model):
    vec = np.zeros(Size).reshape((1,Size))
    count = 0.

    for word in Text:
        try:
            vec += Word2Vec_Model[word].reshape((1,Size))
            count += 1.
        except KeyError:
            continue
    
    if count != 0:
        vec /= count
    
    return vec

RAWDATA 

In [129]:
#getting data from dwh
SupportRawDataframe = getDwhData(CREDENTIALS)
SupportRawDataframe.shape

Downloading: 100%|███████████████████████████████████████████████████████████| 23450/23450 [00:10<00:00, 2336.21rows/s]


(23450, 4)

In [130]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag
0,\nChat transcript:\nVisitor: I want to buy wit...,chat,ps,0
1,\nChat transcript:\nVisitor: hell i had a prob...,chat,ps,0
2,\nChat transcript:\nVisitor: لا استطيع الشراء ...,chat,ps,0
3,\nChat transcript:\nVisitor: im having trouble...,chat,ps,0
4,\nChat transcript:\nVisitor: Hi\nAna: Hello. H...,chat,ps,0
5,\nChat transcript:\nVisitor: Здраствуйте\nAna:...,chat,ps,0
6,\nChat transcript:\nVisitor: hello\nVisitor: w...,chat,ps,0
7,\nChat transcript:\nVisitor: not letting me bu...,chat,ps,0
8,\nChat transcript:\nVisitor: مرحبا\nAna: Hello...,chat,ps,0
9,\nChat transcript:\nVisitor: hi im trying to b...,chat,ps,0


In [131]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag
23440,"\nChat transcript:\nVisitor: Hello, I bought p...",chat,other,1
23441,\nChat transcript:\nVisitor: Good day\n\nI acc...,facebook,other,1
23442,\nChat transcript:\nVisitor: payment not going...,chat,other,1
23443,\nChat transcript:\nVisitor: Paid for faceit s...,chat,other,1
23444,"\nChat transcript:\nVisitor: Hello, I am tryin...",chat,other,1
23445,"\nChat transcript:\nVisitor: Hi, i made a pruc...",chat,other,1
23446,"\nChat transcript:\nVisitor: Hi, how long will...",chat,other,1
23447,\nChat transcript:\nVisitor: I bought playerun...,chat,other,1
23448,\nChat transcript:\nVisitor: Good day i took t...,chat,other,1
23449,\nChat transcript:\nVisitor: hi\nVisitor: hell...,chat,other,1


DATA PREPROC

In [132]:
#transform text to lower case
corpus = SupportRawDataframe.description
corpus.astype('str')

corpus = lowerCase(corpus)

#getting language for text corpus
corpus_lang = getTextLanguage(corpus)

In [133]:
#new dataframe with texts in lower case, without /n symbol and with lang for text
SupportRawDataframe['description'] = corpus
SupportRawDataframe['lang'] = corpus_lang

In [134]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
5,chat transcript:visitor: здраствуйтеana: здрав...,chat,ps,0,ru
6,chat transcript:visitor: hellovisitor: why i c...,chat,ps,0,en
7,chat transcript:visitor: not letting me buy an...,chat,ps,0,en
8,chat transcript:visitor: مرحباana: hello!visit...,chat,ps,0,en
9,chat transcript:visitor: hi im trying to buy v...,chat,ps,0,en


In [135]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag,lang
23440,"chat transcript:visitor: hello, i bought pubg ...",chat,other,1,en
23441,chat transcript:visitor: good dayi accidentall...,facebook,other,1,en
23442,chat transcript:visitor: payment not going thr...,chat,other,1,en
23443,chat transcript:visitor: paid for faceit subsc...,chat,other,1,en
23444,"chat transcript:visitor: hello, i am trying to...",chat,other,1,en
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en
23449,chat transcript:visitor: hivisitor: hellovisit...,chat,other,1,en


In [136]:
#getting only en texts
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
...,...,...,...,...,...
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en


In [137]:
#text tekenization
tokenization = textToTokens(SupportDataframe_eng.description)
SupportDataframe_eng['description'] = tokenization

In [138]:
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[chat, transcript:visitor:, i, want, to, buy, ...",chat,ps,0,en
1,"[chat, transcript:visitor:, hell, i, had, a, p...",chat,ps,0,en
2,"[chat, transcript:visitor:, لا, استطيع, الشراء...",chat,ps,0,en
3,"[chat, transcript:visitor:, im, having, troubl...",chat,ps,0,en
4,"[chat, transcript:visitor:, hiana:, hello., ho...",chat,ps,0,en
...,...,...,...,...,...
23445,"[chat, transcript:visitor:, hi,, i, made, a, p...",chat,other,1,en
23446,"[chat, transcript:visitor:, hi,, how, long, wi...",chat,other,1,en
23447,"[chat, transcript:visitor:, i, bought, playeru...",chat,other,1,en
23448,"[chat, transcript:visitor:, good, day, i, took...",chat,other,1,en


In [139]:
texts = list(SupportDataframe_eng.description)

In [140]:
testdf = SupportDataframe_eng.copy()

1) Осчистка мусора. Частотный анализ.

In [141]:
SupportDataframe_eng['description']

0        [chat, transcript:visitor:, i, want, to, buy, ...
1        [chat, transcript:visitor:, hell, i, had, a, p...
2        [chat, transcript:visitor:, لا, استطيع, الشراء...
3        [chat, transcript:visitor:, im, having, troubl...
4        [chat, transcript:visitor:, hiana:, hello., ho...
                               ...                        
23445    [chat, transcript:visitor:, hi,, i, made, a, p...
23446    [chat, transcript:visitor:, hi,, how, long, wi...
23447    [chat, transcript:visitor:, i, bought, playeru...
23448    [chat, transcript:visitor:, good, day, i, took...
23449    [chat, transcript:visitor:, hivisitor:, hellov...
Name: description, Length: 16003, dtype: object

In [142]:
c = collect.Counter()
for bag in SupportDataframe_eng['description']:
    for word in bag:
        c[word] += 1

In [143]:
dictlist = dict(c)

In [144]:
diclistsort = sorted(dictlist.items(), key=lambda x: x[1], reverse=True)

In [145]:
diclistsort

[('the', 107561),
 ('you', 87781),
 ('i', 86063),
 ('to', 78202),
 ('for', 59021),
 ('a', 58283),
 ('your', 48004),
 ('and', 41004),
 ('it', 36193),
 ('thank', 33741),
 ('please', 29786),
 ('is', 29596),
 ('visitor:', 29301),
 ('have', 28785),
 ('payment', 27003),
 ('this', 26480),
 ('will', 26132),
 ('with', 22596),
 ('that', 22257),
 ('can', 21943),
 ('in', 21636),
 ('my', 21210),
 ('we', 20964),
 ('not', 20717),
 ('on', 19675),
 ('be', 18255),
 ('of', 17785),
 ('refund', 16779),
 ('chat', 16572),
 ('transcript:visitor:', 15782),
 ('was', 15669),
 ('if', 13733),
 ('are', 13552),
 ('our', 13351),
 ('contact', 12994),
 ('but', 11989),
 ('transaction', 11354),
 ('as', 11008),
 ('provide', 10994),
 ('me', 10591),
 ('or', 10442),
 ('do', 10043),
 ('us', 9870),
 ('email', 9347),
 ('may', 9057),
 ('game', 8918),
 ('from', 8513),
 ('page', 8358),
 ('take', 8343),
 ('url:', 8260),
 ('new', 7970),
 ('wait', 7964),
 ('check', 7889),
 ('so', 7815),
 ('help', 7795),
 ('number', 7344),
 ('there', 

In [146]:
wordlistforclean = [] # Добавление stop_words из библиотеки nltk уменьшило f1-score с 68/90 до 65/92. Убрал.
for key in diclistsort[:30]: # Значение 30 - не случайно, получилось так, что хуже и больльше, и меньше. +-5
    wordlistforclean.append(key[0])
for dic in diclistsort:
    if ":" in dic[0]: # Добавлял все знаки пунктуации из string.punctuation, но становилось хуже. Не понятно почему.
            wordlistforclean.append(dic[0])

In [147]:
wordlistforclean

['the',
 'you',
 'i',
 'to',
 'for',
 'a',
 'your',
 'and',
 'it',
 'thank',
 'please',
 'is',
 'visitor:',
 'have',
 'payment',
 'this',
 'will',
 'with',
 'that',
 'can',
 'in',
 'my',
 'we',
 'not',
 'on',
 'be',
 'of',
 'refund',
 'chat',
 'transcript:visitor:',
 'visitor:',
 'transcript:visitor:',
 'url:',
 'may:',
 ':)',
 ']visitor:',
 'you?visitor:',
 'youvisitor:',
 'julia:',
 'you.visitor:',
 'you!visitor:',
 'ismael:',
 'ana:',
 'hellovisitor:',
 'patience.visitor:',
 'okvisitor:',
 'andrew:',
 'lily:',
 'https://help.xsolla.com/',
 'hailey:',
 '?visitor:',
 'else?visitor:',
 'zach:',
 ':)visitor:',
 'joel:',
 'file:',
 'yesvisitor:',
 'hivisitor:',
 ':',
 'itvisitor:',
 'zoey:',
 'this.visitor:',
 'including:',
 'melissa:',
 'request.]visitor:',
 'addressvisitor:',
 'hello.zach:',
 'days:1.',
 'pleasevisitor:',
 'payment.visitor:',
 'michael:',
 'hello!ana:',
 'hello.visitor:',
 'lisa:',
 'hello!visitor:',
 'april:',
 'accountvisitor:',
 'paymentvisitor:',
 '감사합니다.visitor:',

In [148]:
len(wordlistforclean)

60567

In [149]:
#newmas = FastClear(SupportDataframe_eng['description'], wordlistforclean)

In [150]:
newmas = FastClear2(SupportDataframe_eng['description'], wordlistforclean) # Работает значительно быстрее

Done: 500 out of 16003
Done: 1000 out of 16003
Done: 1500 out of 16003
Done: 2000 out of 16003
Done: 2500 out of 16003
Done: 3000 out of 16003
Done: 3500 out of 16003
Done: 4000 out of 16003
Done: 4500 out of 16003
Done: 5000 out of 16003
Done: 5500 out of 16003
Done: 6000 out of 16003
Done: 6500 out of 16003
Done: 7000 out of 16003
Done: 7500 out of 16003
Done: 8000 out of 16003
Done: 8500 out of 16003
Done: 9000 out of 16003
Done: 9500 out of 16003
Done: 10000 out of 16003
Done: 10500 out of 16003
Done: 11000 out of 16003
Done: 11500 out of 16003
Done: 12000 out of 16003
Done: 12500 out of 16003
Done: 13000 out of 16003
Done: 13500 out of 16003
Done: 14000 out of 16003
Done: 14500 out of 16003
Done: 15000 out of 16003
Done: 15500 out of 16003
Done: 16000 out of 16003


In [151]:
SupportDataframe_eng['description'] = newmas
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"{try, ok, request.][visitor, possible?[visitor...",chat,ps,0,en
1,"{hell, .uk, take, his, patience., by, been, cl...",chat,ps,0,en
2,"{وسوف, account, try, purchase., ok, company., ...",chat,ps,0,en
3,"{unable, security, changed, (less, code, than,...",chat,ps,0,en
4,"{becouse, option, ok, purchase., can't, unfort...",chat,ps,0,en
...,...,...,...,...,...
23445,"{already, currency., ingame, xsolla., you,, si...",chat,other,1,en
23446,"{number, details, delivers, although, out, tak...",chat,other,1,en
23447,"{number, details, digits, game, welcome!, requ...",chat,other,1,en
23448,"{things, take, want, or, you., page, cases, ni...",chat,other,1,en


TRAIN TEST SPLIT DATAFRAMES

In [152]:
#list of unique categories
unique_categories = np.unique(SupportDataframe_eng.category)
unique_categories

array(['other', 'ps'], dtype=object)

In [153]:
descriptions = SupportDataframe_eng['description']
categories = SupportDataframe_eng['category_flag']
XTrain, XTest, YTrain, YTest = train_test_split(descriptions,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

FROM TEXTS TO VECTORS

In [154]:
#initialize Word2Vec model for embedding words to vectors
NDim = 100
Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
Imdb_w2v.build_vocab(XTrain)

In [155]:
Imdb_w2v.train(XTrain,total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)

(4234460, 4990675)

In [156]:
#Vecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in descriptions])

In [157]:
#embedding training messages to vectors for neutral classifier
TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTrain])

In [158]:
TrainVecs

array([[ 0.14945012, -0.13114705,  0.15811513, ...,  0.19292204,
         0.27686619, -0.30567972],
       [-0.39370041, -0.28547679, -0.4542865 , ...,  0.28220745,
        -0.39316901,  0.30580376],
       [-0.02244937, -0.24784077, -0.06523112, ...,  0.60024127,
         0.13280638,  0.00662808],
       ...,
       [-0.16215912, -0.38402882, -0.0128114 , ...,  0.6002747 ,
         0.20704153,  0.01704032],
       [-0.09200615, -0.15433143, -0.47526679, ...,  0.35300308,
        -0.45299568,  0.32487484],
       [-0.01652371, -0.13890107, -0.09573785, ...,  0.30222286,
        -0.08984169,  0.22685772]])

In [159]:
Imdb_w2v.train(XTest, total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)

(1053460, 1247660)

In [160]:
TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTest])

In [161]:
TestVecs

array([[-0.31798882, -0.02627644, -0.06597178, ...,  0.23360845,
        -0.13001722,  0.3705747 ],
       [-0.21056817, -0.23490778, -0.24278507, ...,  0.31622431,
         0.02842885,  0.15745146],
       [ 0.0843149 , -0.15906107,  0.06388755, ...,  0.26067282,
         0.2868574 ,  0.07548145],
       ...,
       [-0.16428456,  0.01600793, -0.35333041, ...,  0.20413095,
        -0.33210903,  0.29116047],
       [-0.07162936, -0.2410392 , -0.18013946, ...,  0.20892283,
         0.1039358 ,  0.12984777],
       [ 0.05553926, -0.14222758,  0.00158601, ...,  0.28762773,
         0.3330273 ,  0.13797187]])

CROSSVALIDATION AND BUILD CLASSIFIER

3) Подбор гиперпараметров

In [162]:
TextClassifier = MLPClassifier(hidden_layer_sizes = (30,  2), activation = 'tanh', max_iter = 50, random_state = 40) # Подбор гиперпараметров

In [163]:
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = 5)
print(Scores)
print(np.mean(Scores))

[0.88559157 0.88832487 0.8796875  0.8796875  0.88789063]
0.8842364127782115


In [164]:
TextClassifier.fit(TrainVecs, YTrain)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=50,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=40, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [165]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 496  139]
 [ 267 2299]]
              precision    recall  f1-score   support

           0       0.65      0.78      0.71       635
           1       0.94      0.90      0.92      2566

    accuracy                           0.87      3201
   macro avg       0.80      0.84      0.81      3201
weighted avg       0.88      0.87      0.88      3201



2) Стратифицированная кросс-валидация

In [166]:
desc = np.concatenate([buildWordVector(i, NDim, Imdb_w2v) for i in descriptions])
skf = StratifiedKFold(n_splits=5, random_state=40)
for train_index, test_index in skf.split(desc, categories):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = desc[train_index], desc[test_index]
    y_train, y_test = categories.iloc[train_index], categories.iloc[test_index]
    TextClassifier.fit(X_train, y_train)
    pred = TextClassifier.predict(X_test)
    print(confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred))

TRAIN: [  636   637   638 ... 16000 16001 16002] TEST: [   0    1    2 ... 5738 5739 5740]
[[ 448  188]
 [ 148 2417]]
              precision    recall  f1-score   support

           0       0.75      0.70      0.73       636
           1       0.93      0.94      0.94      2565

    accuracy                           0.90      3201
   macro avg       0.84      0.82      0.83      3201
weighted avg       0.89      0.90      0.89      3201

TRAIN: [    0     1     2 ... 16000 16001 16002] TEST: [ 636  637  638 ... 8304 8305 8306]
[[ 416  219]
 [ 120 2446]]
              precision    recall  f1-score   support

           0       0.78      0.66      0.71       635
           1       0.92      0.95      0.94      2566

    accuracy                           0.89      3201
   macro avg       0.85      0.80      0.82      3201
weighted avg       0.89      0.89      0.89      3201

TRAIN: [    0     1     2 ... 16000 16001 16002] TEST: [ 1271  1272  1273 ... 10870 10871 10872]
[[ 391  244]


4) Trying TF-IDF

In [176]:
descriptions = SupportDataframe_eng['description']
desc = SupportDataframe_eng['description'].apply(lambda x: " ".join(x))
categories = SupportDataframe_eng['category_flag']
XTrain, XTest, YTrain, YTest = train_test_split(desc,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)
text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('clf', MLPClassifier(hidden_layer_sizes = (30,  2), activation = 'tanh', max_iter = 50, random_state = 40))
                     ])


In [177]:
text_clf.fit(XTrain, YTrain)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(30, 2),
                               learning_rate='constant',
                               learning_ra

In [178]:
#text_clf.fit(desc, list(categories))

In [179]:
pred = text_clf.predict(XTest) # Странно. В прошлый раз было f1-score = 0.74 :D
print(confusion_matrix(YTest, pred))
print(classification_report(YTest,pred))

[[ 424  211]
 [ 171 2395]]
              precision    recall  f1-score   support

           0       0.71      0.67      0.69       635
           1       0.92      0.93      0.93      2566

    accuracy                           0.88      3201
   macro avg       0.82      0.80      0.81      3201
weighted avg       0.88      0.88      0.88      3201

